In [1]:
import numpy as np
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors
import seaborn as sns
import plotly
import plotly.express as px

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

from yellowbrick.cluster import KElbowVisualizer
from mpl_toolkits.mplot3d import Axes3D

import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)



* **ID**: Идентификационный номер покупателя
* **Year_Birth**: Год рождения покупателя
* **Education**: Степень образования покупателя 
* **Marital_Status**: Семейный  статус покупателя
* **Income**: Годовой доход покупателя
* **Kidhome**: Количество детей до 13 лет, проживающих с покупателем
* **Teenhome**: Количество детей с 13 до 18, проживающих с покупателем
* **Dt_Customer**: Дата регистрации покупателя как клиента компании 
* **Recency**: Количество дней с последней покупки клиента
* **Complain**: Бинарный признак наличия жалоб от клиента в течение двух лет

* **MntWines**: Количество денег, которые клиент потратил на вино за последние два года 
* **MntFruits**: Количество денег, которые клиент потратил на фрукты за последние два года
* **MntMeatProducts**: Количество денег, которые клиент потратил на мясо за последние два года
* **MntFishProducts**: Количество денег, которые клиент потратил на рыбу за последние два года
* **MntSweetProducts**: Количество денег, которые клиент потратил на сладости за последние два года
* **MntGoldProds**: Количество денег, которые клиент потратил на золотые украшения за последние два года

* **NumDealsPurchases**: Количество покупок товаров со скидкой
* **AcceptedCmp1**: Бинарный признак того, что клиент был привлечен в 1 маркетинговую кампанию
* **AcceptedCmp2**: Бинарный признак того, что клиент был привлечен в 2 маркетинговую кампанию
* **AcceptedCmp3**: Бинарный признак того, что клиент был привлечен в 3 маркетинговую кампанию
* **AcceptedCmp4**: Бинарный признак того, что клиент был привлечен в 4 маркетинговую кампанию
* **AcceptedCmp5**: Бинарный признак того, что клиент был привлечен в 5 маркетинговую кампанию
* **Response**: Бинарный признак того, что клиент откликнулся на последнюю прошедшую маркетинговую кампанию

* **NumWebPurchases**: Количество покупок через сайт компании 
* **NumCatalogPurchases**: Количество покупок через каталог 
* **NumStorePurchases**: Количество покупок через магазин
* **NumWebVisitsMonth**: Количество посещений сайта компании за последний месяц 

In [3]:
data = pd.read_csv("data/marketing_campaign.csv",sep="\t")
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

**Некоторые первые выводы:**

* Есть пропуски в доходе (`Income`)
* Dt_Customer - признак, который показывает когда клиент попал в базу данных фирмы не является временной меткой, а имеет тип `object`
* В данных присуствуют категориальные признаки, которые надо будет закодировать. 

Для простоты, удалим все данные с пропусками.

In [5]:
data = data.dropna()
print(f"Количество объектов после удаления пропусков: {data.shape[0]}")

Количество объектов после удаления пропусков: 2216


In [7]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"]) 
dt_max = data["Dt_Customer"].max()
dt_min = data["Dt_Customer"].min()
print(f"Дата добавления последнего покупателя в базу: {dt_max}")
print(f"Дата добавления первого покупателя в базу: {dt_min}")

Дата добавления последнего покупателя в базу: 2014-12-06 00:00:00
Дата добавления первого покупателя в базу: 2012-01-08 00:00:00
